In [46]:
import pandas as pd
import numpy as np
import datetime as dt
from collections import Counter


In [60]:
ERRORDICT = {
    # gestiamo tutto quello che riguarda i controlli da fare qui
    # in seguito usiamo la key del dict per creare sia le checkbox
    # sia per chiamare le funzioni, sia per creare la colonna bool
    # per la tabella con soli errori
    #
    # in sostanza: vogliamo che ERRORDICT sia il riferimento unico
    # per tutti i controlli che facciamo
    #
    # basta aggiungere qui un nuovo errore e la descrizione
    # e i checkbox etc vengono creati automaticamente
    "errCodFisc1": "Controllo formato codice fiscale",
    "errCodFisc2": "Controllo data nascita per codice fiscale",
    "errAgeChild": "Controllo età bambino",
    "errInizioMinoreFine": "Controllo date contrattuali",
    "errErrorePresenza": "Controllo errore presenza",
    "errErroreDati543": "Controllo errore dati 543",
    "errFineAssistenzaMax4Anni": "Controllo fine contratto assistenza",
    "errKindergarten_1": "Controllo Kindergarten #1",
    "errKindergarten_2": "Controllo Kindergarten #2",
    "errErroreFinanziamentoCompensativo": "Controllo finanziamento compensativo",
    "errErroreFinanziamentoCompensativo2": "Controllo finanziamento compensativo #2",
    "errErroreCovid": "Controllo Covid #1",
    "errErroreCovid2": "Controllo Covid #2",
    "errErroreCovid3": "Controllo Covid #3",
    "errErroreCovid4": "Contratti lockdown sospetti (Covid #4)",
    "errFehlerEingewöhnung": "Controllo Fehler Eingewöhnung",
    "errFehlerEingewöhnung543Lockdown": "Controllo Fehler Eingewöhnung 543 Lockdown",
    "errFehlerEingewöhnung543Notbetreuung": "Controllo Fehler Eingewöhnung 543 Notbetreuung",
    "errGesamtstundenVertragszeitraum": "Controllo ore complessive periodo contrattuale",
    "errSuperatoOreMassime1920": "Controllo ore complessive superiore a 1920",
    "errBambinoInPiuComuni": "Bambino presente in più comuni",
    "errPresentiAnnotazioni": "Bambini con annotazioni",
    "errOreRendicontateZero": "Controllo ore rendicontate uguali a zero",
    "massimo543": "Calcolo valore massimo ore 543",
    "errSommaOre": "Calcolo somme per le ore riportate",
    "errMassimoFC": "Calcolo valore massimo ore finanziamento compensativo",
    "errCalcoloSomme": "Calcolo delle somme ore per comune",
    "errCalcoloNumeroBimbi": "Calcolo numero bambini per comune"
}

In [61]:
fn = "storico.xlsx"

In [62]:
df = pd.read_excel(fn)

In [67]:
dfx = df.groupby("Comune",as_index=False).count()

In [64]:
df.columns

Index(['Unnamed: 0', 'filename', 'Cognome e nome bambino', 'Ente', 'Comune',
       'Data di nascita', 'Codice fiscale',
       'Assistente domiciliare all'infanzia',
       'Comune di residenza assistente domiciliare all'infanzia',
       'Data inizio contratto (o data inizio assistenza se diversa)',
       'GiorniAssistenzaAnnoRiferimento',
       'Data fine contratto\n(o data fine assistenza se diversa) *',
       'Ore di assistenza \n ai sensi della delibera\nn. 666/2019',
       'Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020',
       'massimo543',
       'Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020',
       'Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)',
       'Ore totali rendicontate per il 2020', 'errCodFisc1', 'errCodFisc2',
       'errAgeChild', 'errInizioMinoreFine', 'errErrorePresenza',
       'errErroreDati543', 'errFineAssistenzaMax4Anni', 'errKindergarten_1',
       'errKindergarte

In [65]:
for k in ERRORDICT:
    df = df.drop([k],axis=1)

KeyError: "['errCalcoloNumeroBimbi'] not found in axis"

In [66]:
df.columns

Index(['Unnamed: 0', 'filename', 'Cognome e nome bambino', 'Ente', 'Comune',
       'Data di nascita', 'Codice fiscale',
       'Assistente domiciliare all'infanzia',
       'Comune di residenza assistente domiciliare all'infanzia',
       'Data inizio contratto (o data inizio assistenza se diversa)',
       'GiorniAssistenzaAnnoRiferimento',
       'Data fine contratto\n(o data fine assistenza se diversa) *',
       'Ore di assistenza \n ai sensi della delibera\nn. 666/2019',
       'Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020',
       'Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020',
       'Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)',
       'Ore totali rendicontate per il 2020', 'gg1', 'gg2', 'gg3',
       'inizioNorm', 'fineNorm'],
      dtype='object')

In [72]:
df

,Unnamed: 0,filename,Cognome e nome bambino,Ente,Comune,Data di nascita,Codice fiscale,Assistente domiciliare all'infanzia,Comune di residenza assistente domiciliare all'infanzia,Data inizio contratto (o data inizio assistenza se diversa),...,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo),Ore totali rendicontate per il 2020,gg1,gg2,gg3,inizioNorm,fineNorm
0,11,Erklärung_Gemeinde_Algund_2020.xlsx,FLARER THEO,Sozialgenossenschaft Tagesmütter - Gen.,Algund,2018-07-11,FLRTHE18L11F132U,LADURNER JOHANNA,Meran/Merano,2020-03-02,...,459.00,337.00,0,0.5,796.50,74,8,0,2020-03-02,2020-12-31
1,16,Erklärung_Gemeinde_Algund_2020.xlsx,MIRANDOLA LENZ,Sozialgenossenschaft Tagesmütter - Gen.,Algund,2019-03-17,MRNLNZ19C17F132A,BRUNELLO CHRISTINE,Algund/Lagundo,2020-11-09,...,74.50,23.75,0,0.0,98.25,0,8,0,2020-11-09,2020-12-31
2,8,Erklärung_Gemeinde_Branzoll_2020.xlsx,CABULA LEONARDO,Sozialgenossenschaft Tagesmütter - Gen.,Branzoll,2017-07-24,CBLLRD17L24A952H,PUFF CHRISTINA,Leifers/Laives,2020-01-27,...,1199.50,348.00,0,4.0,1551.50,74,8,2,2020-01-27,2020-12-31
3,9,Erklärung_Gemeinde_Brixen_2020.xlsx,ALBERTI VICKY,Sozialgenossenschaft Tagesmütter - Gen.,Brixen,2017-11-03,LBRVKY17S43B160U,PUNER PAULA,Vahrn/Varna,2018-09-03,...,884.25,234.00,0,24.5,1142.75,74,8,0,2020-01-01,2020-12-31
4,23,Erklärung_Gemeinde_Brixen_2020.xlsx,NITZ MIA,Sozialgenossenschaft Tagesmütter - Gen.,Brixen,2017-10-06,NTZMIA17R46B160F,PUNER PAULA,Vahrn/Varna,2018-09-24,...,868.50,296.00,0,29.0,1193.50,74,8,0,2020-01-01,2020-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,10,Erklärung_Gemeinde_Terenten_2020.xlsx,INNERHOFER JULIAN,Sozialgenossenschaft Tagesmütter - Gen.,Terenten,2017-01-17,NNRJLN17A17B220Y,LECHNER MARTHA,Terenten/Terento,2018-01-08,...,208.00,406.00,0,0.0,614.00,74,0,0,2020-01-01,2020-06-26
92,8,Erklärung_Gemeinde_Tscherms_2020.xlsx,MAIRHOFER MARIE,Sozialgenossenschaft Tagesmütter - Gen.,Tscherms,2017-10-05,MRHMRA17R45F132N,LAGEDER JOHANNA,Lana/Lana,2019-10-07,...,394.00,10.00,0,0.0,404.00,74,8,0,2020-01-01,2020-12-31
93,10,Erklärung_Gemeinde_Völs am Schlern_2020.xlsx,BAUMGARTNER PAUL //,Sozialgenossenschaft Tagesmütter - Gen.,Völs am Schlern,2019-09-29,BMGPLA19P29A952I,HASELRIEDER ANGELIKA,Völs am Schlern/Fie' allo Sciliar,2020-12-21,...,13.50,0.00,0,0.0,13.50,0,0,0,2020-12-21,2020-12-31
94,13,Erklärung_Gemeinde_Völs am Schlern_2020.xlsx,HOFER HANNAH,Sozialgenossenschaft Tagesmütter - Gen.,Völs am Schlern,2017-09-25,HFRHNH17P65B160X,HASELRIEDER ANGELIKA,Völs am Schlern/Fie' allo Sciliar,2018-04-23,...,335.25,120.00,0,-11.5,443.75,74,0,0,2020-01-01,2020-09-18


In [118]:
df1 = df.groupby('Comune',as_index=False)['Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020'].apply(lambda x: x[x > 0].count())

In [119]:
df1

,Comune,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020
0,Algund,2
1,Bozen,14
2,Branzoll,1
3,Brixen,2
4,Bruneck,1
5,Eppan a.d. Weinstr.,3
6,Freienfeld,2
7,Gargazon,0
8,Klausen,0
9,Laas,1


In [116]:
df2 = df.groupby('Comune',as_index=False)['Ore di assistenza \n ai sensi della delibera\nn. 666/2019'].apply(lambda x: x[x > 0].count())

In [117]:
df2

,Comune,Ore di assistenza \n ai sensi della delibera\nn. 666/2019
0,Algund,2
1,Bozen,17
2,Branzoll,1
3,Brixen,2
4,Bruneck,2
5,Eppan a.d. Weinstr.,4
6,Freienfeld,2
7,Gargazon,1
8,Klausen,1
9,Laas,1


In [122]:
df3 = df.groupby('Comune',as_index=False)['Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020'].apply(lambda x: x[x > 0].count())

In [130]:
df4 = df.groupby('Comune',as_index=False)['Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)'].apply(lambda x: x[x > 0].count())

In [131]:
df4

,Comune,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)
0,Algund,1
1,Bozen,10
2,Branzoll,1
3,Brixen,2
4,Bruneck,1
5,Eppan a.d. Weinstr.,3
6,Freienfeld,2
7,Gargazon,0
8,Klausen,0
9,Laas,1


In [120]:
dfx = pd.merge(df1, df2, on='Comune', how='outer')

In [123]:
dfx = pd.merge(dfx, df3, on='Comune', how='outer')

In [124]:
dfx

,Comune,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020
0,Algund,2,2,0
1,Bozen,14,17,0
2,Branzoll,1,1,0
3,Brixen,2,2,0
4,Bruneck,1,2,0
5,Eppan a.d. Weinstr.,3,4,0
6,Freienfeld,2,2,0
7,Gargazon,0,1,0
8,Klausen,0,1,0
9,Laas,1,1,0


In [125]:
df

,Unnamed: 0,filename,Cognome e nome bambino,Ente,Comune,Data di nascita,Codice fiscale,Assistente domiciliare all'infanzia,Comune di residenza assistente domiciliare all'infanzia,Data inizio contratto (o data inizio assistenza se diversa),...,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo),Ore totali rendicontate per il 2020,gg1,gg2,gg3,inizioNorm,fineNorm
0,11,Erklärung_Gemeinde_Algund_2020.xlsx,FLARER THEO,Sozialgenossenschaft Tagesmütter - Gen.,Algund,2018-07-11,FLRTHE18L11F132U,LADURNER JOHANNA,Meran/Merano,2020-03-02,...,459.00,337.00,0,0.5,796.50,74,8,0,2020-03-02,2020-12-31
1,16,Erklärung_Gemeinde_Algund_2020.xlsx,MIRANDOLA LENZ,Sozialgenossenschaft Tagesmütter - Gen.,Algund,2019-03-17,MRNLNZ19C17F132A,BRUNELLO CHRISTINE,Algund/Lagundo,2020-11-09,...,74.50,23.75,0,0.0,98.25,0,8,0,2020-11-09,2020-12-31
2,8,Erklärung_Gemeinde_Branzoll_2020.xlsx,CABULA LEONARDO,Sozialgenossenschaft Tagesmütter - Gen.,Branzoll,2017-07-24,CBLLRD17L24A952H,PUFF CHRISTINA,Leifers/Laives,2020-01-27,...,1199.50,348.00,0,4.0,1551.50,74,8,2,2020-01-27,2020-12-31
3,9,Erklärung_Gemeinde_Brixen_2020.xlsx,ALBERTI VICKY,Sozialgenossenschaft Tagesmütter - Gen.,Brixen,2017-11-03,LBRVKY17S43B160U,PUNER PAULA,Vahrn/Varna,2018-09-03,...,884.25,234.00,0,24.5,1142.75,74,8,0,2020-01-01,2020-12-31
4,23,Erklärung_Gemeinde_Brixen_2020.xlsx,NITZ MIA,Sozialgenossenschaft Tagesmütter - Gen.,Brixen,2017-10-06,NTZMIA17R46B160F,PUNER PAULA,Vahrn/Varna,2018-09-24,...,868.50,296.00,0,29.0,1193.50,74,8,0,2020-01-01,2020-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,10,Erklärung_Gemeinde_Terenten_2020.xlsx,INNERHOFER JULIAN,Sozialgenossenschaft Tagesmütter - Gen.,Terenten,2017-01-17,NNRJLN17A17B220Y,LECHNER MARTHA,Terenten/Terento,2018-01-08,...,208.00,406.00,0,0.0,614.00,74,0,0,2020-01-01,2020-06-26
92,8,Erklärung_Gemeinde_Tscherms_2020.xlsx,MAIRHOFER MARIE,Sozialgenossenschaft Tagesmütter - Gen.,Tscherms,2017-10-05,MRHMRA17R45F132N,LAGEDER JOHANNA,Lana/Lana,2019-10-07,...,394.00,10.00,0,0.0,404.00,74,8,0,2020-01-01,2020-12-31
93,10,Erklärung_Gemeinde_Völs am Schlern_2020.xlsx,BAUMGARTNER PAUL //,Sozialgenossenschaft Tagesmütter - Gen.,Völs am Schlern,2019-09-29,BMGPLA19P29A952I,HASELRIEDER ANGELIKA,Völs am Schlern/Fie' allo Sciliar,2020-12-21,...,13.50,0.00,0,0.0,13.50,0,0,0,2020-12-21,2020-12-31
94,13,Erklärung_Gemeinde_Völs am Schlern_2020.xlsx,HOFER HANNAH,Sozialgenossenschaft Tagesmütter - Gen.,Völs am Schlern,2017-09-25,HFRHNH17P65B160X,HASELRIEDER ANGELIKA,Völs am Schlern/Fie' allo Sciliar,2018-04-23,...,335.25,120.00,0,-11.5,443.75,74,0,0,2020-01-01,2020-09-18


In [126]:
df4 = df.groupby('Comune',as_index=False)['Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)'].apply(lambda x: x[x > 0].count())

In [127]:
df4

,Comune,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)
0,Algund,1
1,Bozen,10
2,Branzoll,1
3,Brixen,2
4,Bruneck,1
5,Eppan a.d. Weinstr.,3
6,Freienfeld,2
7,Gargazon,0
8,Klausen,0
9,Laas,1


In [128]:
dfx = pd.merge(dfx, df4, on='Comune', how='outer')

In [129]:
dfx

,Comune,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)
0,Algund,2,2,0,1
1,Bozen,14,17,0,10
2,Branzoll,1,1,0,1
3,Brixen,2,2,0,2
4,Bruneck,1,2,0,1
5,Eppan a.d. Weinstr.,3,4,0,3
6,Freienfeld,2,2,0,2
7,Gargazon,0,1,0,0
8,Klausen,0,1,0,0
9,Laas,1,1,0,1
